# 构建具备自主能力的 AI 应用

---

既然你已经掌握了检索增强生成（RAG），现在是迈向下一步的时候了：Agentic AI。

RAG 强化了大语言模型的知识广度，而智能体则增强了它的行动能力。AI 智能体不只是一个聊天机器人，它是能够感知、规划、行动并反思以达成目标的自主系统。

在本章中，你将学习如何构建超越回答问题的智能体——它们会执行任务、调用工具并制定决策。

我们会将 **TaskFriend** 从一个被动助手扩展成积极主动的智能体，帮助用户规划行程、查看日程并获取天气信息——这一切都依赖多个专门智能体的协同编排。

让我们先弄清楚，是什么让一个 AI 系统具备“智能体”特性。



## 目前的故事进展……

**TaskFriend** 正在全球各大应用商店和市场中快速攀升排行榜。但庞大的用户群也带来了新的责任：

> 用户希望能做得更多——他们不只想规划一天的日程，还想围绕自己的时间安排规划**旅行**

你注意到，以前的优化技巧已经无法轻松扩展新功能：

* 继续堆叠系统提示和模板会让系统更脆弱
* 用户提的问题越多，底层 LLM 的专精程度越差
* 你不能再只依赖现有的知识库
* 需要实时数据输入
* 随着成本上升，必须优化 LLM 的使用方式

## 目标

* 将复杂目标拆解成具体步骤
* 创建能够基于用户输入做出决策的智能体
* 编排多个专门智能体协同工作
* 使用工具和 API 获取实时数据
* 将结果整合成连贯且易于理解的方案


## 初始化环境

### 设置 API 密钥

在任何笔记本中开始工作前，我们都需要加载 [Model Studio 的 API 密钥](https://modelstudio.console.alibabacloud.com/?tab=globalset#/efm/api_key)。这样才能调用本课程中将使用的 Qwen 模型。

> 如果你还不确定如何找到 **Model Studio** 的 API 密钥，请参考 `00 Setting Up the Environment` 文件。

In [1]:
# Load Model Studio API key
import os
from config.load_key import load_key
load_key(
    confirmation=False
)

Detected existing API key: sk-8...2716
Auto-confirmation enabled. Using existing API key.


In [2]:
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)
            
def get_qwen_stream_response(query, print_stream=True):
    completion = client.chat.completions.create(
        model="qwen-plus",
        messages=[{"role": "user", "content": query}],
        stream=True
    )
    full_response = ""
    for chunk in completion:
        try:
            content = chunk.choices[0].delta.content
            if content:
                if print_stream:
                    print(content, end="", flush=True)
                full_response += content
        except Exception as e:
            print(f"[Stream error: {e}]", flush=True)
    return full_response

# 认识智能体框架

---

一个 AI 智能体由四个核心组件构成：

<div style="text-align: center;">
  <img src="images/LMP-C01_07-Agent.jpeg" style="max-width: 800px;" />
  <br>
  <small>智能体的组成元素</small>
  <br>
</div>

1. 规划（Planning）
    * 智能体会把目标拆解成多个步骤。
    * 使用推理（例如 chain-of-thought）来决定下一步行动。
    * 示例："用户想规划一次旅行 → 首先检查日程是否有空档。"
2. 记忆（Memory）
    * 短期记忆：对话历史（存放在线程中）。
    * 长期记忆：包含用户偏好、过往行程等内容的向量数据库。
    * 支撑个性化体验与持续对话。
3. 行动（工具调用）
    * 智能体通过工具（函数）行动，例如 check_availability、get_weather。
    * 工具连接外部系统（API、数据库）。
    * 智能体负责判断何时以及如何使用工具。
4. 执行与反思
    * 在行动之后，智能体会评估结果。
    * 它可以重试、请求澄清或调整计划。


# 构建你的第一个智能体

---

借助 **Model Studio Assistant API** 构建一个智能体只需四个步骤：

```mermaid
graph LR
    subgraph agent[智能体创建流程]
        A[定义目标] --> B[创建自定义函数]
        B --> C[集成函数]
        C --> D[测试智能体]
    end
```

## 步骤 1：定义智能体目标

智能体最吸引人的特性在于它们各自都是**专家**。也就是说，每个智能体都是围绕单一、清晰目标的**专才**。在这里，我们将扩展 **TaskFriend** 的能力，让它协助规划旅行（谁说任务管理只能限于工作呢？）

```mermaid
graph TD
    subgraph Intent[意图分类]
    direction LR
        A["用户查询"] --> C{"是否已知<br>目的地？"}
        C -->|否| E["旅行顾问智能体"]
    end
    C -->|是| D["规划智能体"]
    E --> D
    subgraph Planner
    direction LR
        D --> H1["日历智能体"]
        D --> H2["天气智能体"]
        D --> Hx["..."]
        H1 --> I["汇总智能体"]
        H2 --> I
        Hx --> I
        I --> J["最终行程规划"]
    end
```

不过先别急——在规划旅行之前，最重要的是确认你的日程是否空闲。所以我们先从创建 `calendar_agent` 开始！



## 步骤 2：创建自定义函数

从零开始打造一个智能体并不容易，因此阿里云通过 **Model Studio** 提供了 **Assistant API**，帮助你简化流程。不过在深入 **Assistant API** 之前，我们先编写一个自定义函数，来执行 `calendar_agent` 的职责。

要让 `calendar_agent` 正常工作，需要具备哪些能力？

* 能访问用户日历
* 了解当前及未来的日期
* 可以检查用户是否有空

> 在这个例子里，我们虽然使用 `tzlocal` 获取系统时间，但会模拟一些我们不空闲的日期。

In [3]:
from datetime import datetime, timedelta
from tzlocal import get_localzone

def check_availability(date_range="upcoming weekend"):
    """
    Check user's calendar availability using the machine's local timezone.
    """
    # Get the system's local timezone automatically
    user_tz = get_localzone()
    now = datetime.now(user_tz)

    # Calculate upcoming Saturday and Sunday
    days_ahead = 5 - now.weekday()  # Saturday is weekday 5
    if days_ahead <= 0:
        days_ahead += 7
    saturday = now.date() + timedelta(days=days_ahead)
    sunday = saturday + timedelta(days=1)

    # Simulated busy check
    busy_dates = [
        datetime(2025, 7, 4).date(),  # Example holiday
    ]

    available_days = []
    busy_events = []

    for day in [saturday, sunday]:
        if day in busy_dates:
            busy_events.append(f"Blocked: Holiday on {day}")
        else:
            available_days.append(day.strftime("%A, %B %d"))

    return {
        "available_days": available_days,
        "busy_events": busy_events,
        "checked_range": {
            "start": saturday.isoformat(),
            "end": sunday.isoformat()
        },
        "timezone": str(user_tz)
    }

## 步骤 3：将函数集成进智能体

既然函数已经准备就绪，现在是时候把它封装到 **Assistant API** 中。为此，我们首先必须向助手描述这个函数，以便它能够正确调用。

这一步通过定义一个 `tool`（在这里是 `calendar_tool`）完成。它遵循 **Assistant API** 规定的严格模式，后文会详细介绍这一模式。

In [4]:
import dashscope
from dashscope import Assistants, Threads, Messages, Runs
import json

# Define the tool schema
calendar_tool = {
    "type": "function",
    "function": {
        "name": "check_availability",
        "description": "Check if the user is free on upcoming weekend days. Returns available days and any busy events.",
        "parameters": {
            "type": "object",
            "properties": {
                "date_range": {
                    "type": "string",
                    "description": "The date range to check, e.g. 'upcoming weekend', 'next Saturday'"
                }
            },
            "required": ["date_range"]
        }
    }
}

定义好工具后，我们就可以使用 `Assistants.create` 函数，把自定义代码集成进智能体应用。

In [5]:
# Create the agent

dashscope.base_http_api_url ="https://dashscope.aliyuncs.com/api/v1"

calendar_agent = Assistants.create(
    model="qwen-plus",
    name="Calendar Checker",
    description="An AI agent that checks user's calendar availability for trip planning.",
    instructions="You are a calendar assistant. Use the check_availability function to determine if the user is free on weekend days.",
    tools=[calendar_tool]
)

print(f"✅ {calendar_agent.name} created with ID: {calendar_agent.id}")

✅ Calendar Checker created with ID: asst_aa9a76f1-82fe-4bf5-8129-fd40f91081c8


最后，为了通过 **Assistant API** 获取 **智能体**（或称 *assistant*）的响应，我们需要定义一个函数，负责处理它所依赖的 `thread`、`messages` 和 `runs` 参数：

In [6]:
def get_agent_response(assistant, message=''):
    thread = Threads.create()
    Messages.create(thread.id, content=message)
    run = Runs.create(thread.id, assistant_id=assistant.id)
    run_status = Runs.wait(run.id, thread_id=thread.id)

    # Handle tool calls
    if run_status.required_action:
        tool_call = run_status.required_action.submit_tool_outputs.tool_calls[0]
        func_name = tool_call.function.name
        args = json.loads(tool_call.function.arguments)

        # Execute the correct function
        if func_name == "suggest_destinations":
            result = suggest_destinations(args['query'])
        elif func_name == "get_weather_forecast":
            result = get_weather_forecast(args['location'])
        elif func_name == "check_availability":
            result = check_availability(args.get('date_range', 'upcoming weekend'))
        else:
            result = {"error": "Unknown function"}

        # Submit tool output back to the agent
        Runs.submit_tool_outputs(
            run.id,
            thread_id=thread.id,
            tool_outputs=[{"tool_call_id": tool_call.id, "output": json.dumps(result)}]
        )
        run_status = Runs.wait(run.id, thread_id=thread.id)

    # Get final response
    msgs = Messages.list(thread.id)
    return msgs['data'][0]['content'][0]['text']['value']

## 步骤 4：测试智能体

终于可以测试智能体了！

In [7]:
response = get_agent_response(
    calendar_agent,
    "我正计划下周末进行一次公路旅行。"
)

print(response)

下周末（1月10日星期六和1月11日星期日）您目前没有任何安排，这两天都是空闲的。您可以放心规划公路旅行！需要我帮您进一步安排行程吗？


看起来一切运行良好！接下来进入下一部分。

# 意图分类：弄清用户真正想要什么

---

你已经构建了日历检查器——它在用户想确认自己能否请一两天假时非常好用。然而，有时用户突发奇想想去旅行，却还没决定目的地。此时我们的智能体应用面临分流难题：要把查询交给负责帮忙选目的地的 `travel_advisor`，还是交给负责制定行程的 `trip_planner`？

为了解决这个问题，我们可以构建一个简单的意图分类器，遵循以下规则：

* 如果用户已经给出具体地点，就把请求交给 `trip_planner`
* 如果用户没有指定地点，就交给 `travel_advisor`

我们还添加了一些 few-shot 示例，帮助模型处理复杂提问，例如：

> 从新加坡出发的周末旅行有什么好推荐？

## 构建意图路由器

顾名思义，意图路由器的作用就是：

> 根据判断到的意图，将请求（也就是本例中的查询）转交给不同的智能体处理。

In [8]:
def classify_trip_intent(question: str) -> str:
    prompt = """
        [Role]
        You are a travel intent classifier. Your task is to determine whether the user already knows their travel destination or is still exploring options.

        [Task]
        Classify the user's input into one of the following two categories:
        1. knows_destination
        2. unsure_destination

        [Rules]
        - Output ONLY the category name. Do not include any explanation or formatting.
        - If the user mentions a specific place (e.g., Lake Tahoe, Singapore, Napa, Beijing), output: knows_destination
        - If the user expresses uncertainty, curiosity, or exploration (e.g., "I want to get away", "somewhere scenic"), output: unsure_destination
        - Do not consider conversation history. Classify based solely on the current input.

        [Few-shot examples]
        Example 1: User input: "I want to get away this weekend but don’t know where."
        Classification: unsure_destination

        Example 2: User input: "Plan a trip to Lake Tahoe."
        Classification: knows_destination

        Example 3: User input: "Somewhere with mountains and hiking?"
        Classification: unsure_destination

        Example 4: User input: "Can we go to Yosemite this weekend?"
        Classification: knows_destination

        Example 5: User input: "What are good weekend trips from Singapore?"
        Classification: unsure_destination

        [User input]
        Please classify the following input:
    """

    full_prompt = prompt + f'"{question}"\nClassification:'
    response = get_qwen_stream_response(full_prompt)
    
    # Clean and normalize the response
    cleaned = response.strip().strip('"').strip("'").strip()
    
    if "knows" in cleaned.lower():
        return "knows_destination"
    elif "unsure" in cleaned.lower():
        return "unsure_destination"
    else:
        return "unsure_destination"

## 测试意图路由器

现在路由器已经搭好，该检验它能否准确识别我们预期的行为：

* 当用户已经确定目的地时，路由到 `trip_planner`
* 当用户还拿不定主意时，路由到 `travel_advisor`

In [9]:
from datetime import datetime
from tzlocal import get_localzone

def route_trip_query(query: str):
    intent = classify_trip_intent(query)
    user_tz = get_localzone()
    now = datetime.now(user_tz)
    print(f"\n\n🔍 Identified intent: {intent}\n")

    # Define agent behaviors
    def travel_advisor():
        print("🧠 [travel_advisor] Thinking of getaway options...\n")
        response = get_qwen_stream_response(f"""
            Suggest 3 getaway locations based on {query}.
            If the user does not specify a general location, use the user's timezone: {user_tz}
            to suggest 3 locations in the same timezone.
            If the user specifies a general location, provide feasibility based on 
            the user's timezone: {user_tz}, and suggest 3 locations based on feasibility,
            taking into account travel time to and fro.
            Be short and concise.
        """)

    def trip_planner():
        print("🧠 [planner_agent] Starting full trip planning workflow...\n")
        print("📅 Checking calendar...")
        print("🌤️ Checking weather for your destination...")
        print("🚗 Getting travel time...")

        response = get_qwen_stream_response(f"""
            Provide a plan to the destination stated in {query}.
            Use the following format:
            * Calendar: Free next weekend.
            * Weather: Return average weather condition at destination based on 
              average weather conditions over the years based on {now}.
            * Travel options: Return travel options and estimated duration 
              based on where the user is ({user_tz}).
            Be brief and concise.
        """)

    # Route based on intent
    if intent == "unsure_destination":
        print("📨 Routing to travel_agent...\n")
        travel_advisor()
    elif intent == "knows_destination":
        print("📨 Routing to planner_agent...\n")
        trip_planner()
    else:
        print("I didn't understand your request. Want help planning a trip?")

In [10]:
# Test 1: User unsure
user_unsure="I want to get away this weekend but don’t know where."
print("\n" + "=" * 50)
print(f"Query: {user_unsure}")
print("=" * 50)
route_trip_query(user_unsure)

# Test 2: User knows destination
user_knows="Plan a trip to Lake Tahoe."
print("\n\n" + "=" * 50)
print(f"Query: {user_knows}")
print("=" * 50)
route_trip_query(user_knows)

# Test 3: User knows a rough destination
user_kinda_knows="I've never been to Europe before, what do you suggest for a 3-day trip?"
print("\n\n" + "=" * 50)
print(f"Query: {user_kinda_knows}")
print("=" * 50)
route_trip_query(user_kinda_knows)


Query: I want to get away this weekend but don’t know where.
unsure_destination

🔍 Identified intent: unsure_destination

📨 Routing to travel_agent...

🧠 [travel_advisor] Thinking of getaway options...

1. Hangzhou, China – 1-hour train ride from Shanghai; scenic West Lake and peaceful temples.  
2. Huangshan (Yellow Mountain), China – 3-hour high-speed train; stunning mountain views and hot springs.  
3. Suzhou, China – 30-minute train; classical gardens and ancient canals.  

All within same timezone (Asia/Shanghai) and feasible for a weekend trip.

Query: Plan a trip to Lake Tahoe.
knows_destination

🔍 Identified intent: knows_destination

📨 Routing to planner_agent...

🧠 [planner_agent] Starting full trip planning workflow...

📅 Checking calendar...
🌤️ Checking weather for your destination...
🚗 Getting travel time...
* Calendar: Free next weekend.  
* Weather: Cold and snowy, average temperature around -1°C to 6°C; possible snowfall, especially at higher elevations.  
* Travel opt

很好，既然意图路由器已经奏效，我们的应用就更聪明了一点。这是 **多智能体工作流** 的关键节点示例：我们能够掌控应用使用哪些工具来解决用户面临的挑战。

简而言之，你构建的不再只是聊天机器人，而是一个能够决策、并根据任务类型调整行为的应用。这类工作流带来多项优势：

* 无需依赖单一的系统提示或提示模板覆盖所有情况。每个智能体都可以拥有独立指令。
* 你可以为特定功能打造专用工具：
    * 专用工具更聚焦，因而更容易得到良好结果
    * 专用工具可能更省资源，有助于降低成本
    
好了，既然路由环节搞定，就继续构建我们的 **多智能体工作流** 吧！

# 多智能体工作流：一场协同演出

---

## Planner 智能体

Planner 智能体可以说是多智能体工作流中最核心的组件。它会评估用户查询，并挑选最合适的工具。

让我们先创建一个简单版本，并验证它的效果：

In [11]:
planner_agent = Assistants.create(
    model="qwen-plus",
    name="Trip Strategist",
    description="Plans getaways by coordinating multiple agents at its disposal."
)
print(f"✅ Planner Agent created: {planner_agent.name}")

print(get_agent_response(planner_agent,"I want to plan a trip to Disneyland"))

✅ Planner Agent created: Trip Strategist
Great! I'm excited to help you plan a magical trip to Disneyland! To get us started, could you please tell me:

1. **Which Disneyland** are you planning to visit? (e.g., Disneyland Resort in California, Shanghai Disneyland, Hong Kong Disneyland, etc.)
2. **Travel dates** – Do you have specific dates in mind, or is that flexible?
3. **Duration of stay** – How many days do you plan to spend at the park?
4. **Travelers** – Who's going? (e.g., solo, couple, family with kids, group of friends)
5. **Budget range** – Are you looking for budget-friendly, mid-range, or luxury options?
6. **Interests** – Any must-do attractions, character meetups, dining experiences, or special events?

Once I have these details, I’ll coordinate with my team of agents to craft your perfect itinerary! 🏰✨


接下来，我们希望为 `planner_agent` 添加说明，指导它如何行事：

In [12]:
planner_agent = Assistants.update(
    planner_agent.id,
    instructions="""
            You are TaskFriend, a personal trip planning assistant.
            You have access to these agents:
            - calendar_agent
            - weather_agent
            - travel_agent

            Rules:
            1. If the user doesn't know where to go, use travel_agent FIRST.
            2. Once a destination is chosen, plan the trip in this order:
               - calendar_agent (check availability)
               - weather_agent (check conditions)
            3. Output ONLY a JSON list of agent names in execution order.
               Example: ["calendar_agent", "weather_agent"]
            4. Do not include any explanations.
        """
    )

print(f"✅ Updated Planner Agent instructions: {planner_agent.name}")

✅ Updated Planner Agent instructions: Trip Strategist


In [13]:
test_queries = [
    "I want to get away this weekend but don’t know where.",
    "Plan a trip to Lake Tahoe.",
    "I've never been to France before, what are good spots to see?"
]

for query in test_queries:
    print(f"\n📝 Query: {query}")
    plan = get_agent_response(planner_agent, query)
    print(f"📋 Plan: {plan}")


📝 Query: I want to get away this weekend but don’t know where.
📋 Plan: ["travel_agent"]

📝 Query: Plan a trip to Lake Tahoe.
📋 Plan: ["calendar_agent", "weather_agent"]

📝 Query: I've never been to France before, what are good spots to see?
📋 Plan: ["travel_agent"]


## 工具智能体

随着 Planner 上线，我们还需要创建工作流中的其他工具（智能体）。

### `travel_agent`

首先创建 `travel_agent`。它应该能够推荐附近的短途旅行地点：

* 根据查询中提到的位置进行推断
* 若查询未提到具体地点，则依据你的时区推荐附近地点

In [14]:
def extract_location_with_llm(query: str) -> str:
    """
    Use an LLM to extract the destination or origin from a natural language query.
    Returns 'nearby' if no specific location is mentioned.
    """
    prompt = f"""
        You are a location extraction assistant. Your task is to identify any travel destination, city, region, or point of interest mentioned in the user's query.
        - If a specific location is mentioned (e.g., 'London', 'near Paris', 'from Tokyo'), return just the location name.
        - If no specific location is mentioned (e.g., 'near me', 'somewhere scenic', 'I want to get away'), return "nearby".
        - Do NOT include any explanations or formatting.
        - Output ONLY the location or "nearby".

        User Query: "{query}"

        Location:
    """
    
    # Use your existing LLM interface
    response = get_qwen_stream_response(prompt)  # Uses non-streaming call
    
    # Clean and normalize the response
    cleaned = response.strip().strip('"').strip("'").strip()
    
    # Fallback if the model returns something irrelevant
    if not cleaned or cleaned.lower() in ["none", "unknown", "no location"]:
        return "nearby"
    
    return cleaned

def suggest_destinations(query: str):
    """
    Suggest getaway destinations based on the user's query.
    Uses LLM to extract location and generate suggestions.
    """
    # Get user's current timezone
    user_tz = get_localzone()
    
    # Extract location using LLM (no regex)
    extracted_location = extract_location_with_llm(query)
    
    # Determine origin context
    if extracted_location.lower() != "nearby":
        origin_context = extracted_location
    else:
        origin_context = f"within the {user_tz} timezone"
    
    # Generate suggestions using LLM
    prompt = f"""
        Suggest 2 weekend getaway destinations near {origin_context} within the same timezone ± 1 hour.
        Suggest 1 additional getaway destination further away (can be in a different country or region).
        For each destination, include:
        - Name (e.g., a well-known natural or cultural site)
        - Travel time (e.g., "3h 15m drive" or "2h flight")
        - Why it's a good fit for a weekend trip

        Respond in JSON format:
        {{
          "origin": "{origin_context}",
          "recommended_destinations": [
            {{
              "name": "...",
              "travel_time": "...",
              "reason": "..."
            }}
          ]
        }}
    """
    
    response = get_qwen_stream_response(prompt)
    try:
        # Parse the response as JSON
        import json
        result = json.loads(response)
        return result
    except json.JSONDecodeError as e:
        # If JSON parsing fails, return a clean error structure
        return {
            "error": f"Invalid JSON response: {str(e)}",
            "origin": origin_context,
            "recommended_destinations": []
        }
    except Exception as e:
        return {
            "error": f"Could not parse LLM response: {str(e)}",
            "origin": origin_context,
            "recommended_destinations": []
        }

In [15]:
# Define the tool
travel_tool = {
    "type": "function",
    "function": {
        "name": "suggest_destinations",
        "description": "Suggest weekend getaway destinations based on user query. Uses user's timezone if no location is specified.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The user's natural language request, e.g., 'I want to get away this weekend' or 'Plan a trip near London'"
                }
            },
            "required": ["query"]
        }
    }
}

In [16]:
travel_agent = Assistants.create(
    model="qwen-plus",
    name="Travel Advisor",
    description="An AI agent that suggests weekend getaway destinations based on user intent and location.",
    instructions="""
        You are TaskFriend, a friendly and proactive travel advisor.
        Your job is to help users discover weekend getaway options when they're unsure of where to go.

        You have access to a tool called `suggest_destinations` that can:
        - Extract the location from the user's query (e.g., "near London")
        - If no location is mentioned, use the user's current timezone as the origin
        - Suggest 2 nearby destinations (within ±1 hour timezone)
        - Suggest 1 further destination
        - Return structured JSON with name, travel time, and reason

        Rules:
        1. Always use the `suggest_destinations` tool when the user is exploring or unsure.
        2. Do not make up destinations — rely on the tool.
        3. Present the results clearly and conversationally.
        4. End with: "Which one sounds best, or should I suggest more?"
    """,
        tools=[travel_tool]
    )

print(f"✅ {travel_agent.name} created with ID: {travel_agent.id}")

✅ Travel Advisor created with ID: asst_5b646d4f-229c-4b92-b9d8-2ad9a38b29aa


### `weather_agent`：接入实时天气 API

对于 `weather_agent`，我们将使用开源的 **open-meteo** API。该接口在非商业场景下免费，无需 API 密钥，非常契合本例需求！

In [17]:
import requests
import json

def get_weather_forecast(location: str):
    # 1. Geocode using Open-Meteo's geocoding API (no key needed)
    geo_url = "https://geocoding-api.open-meteo.com/v1/search"
    geo_params = {"name": location, "count": 1}
    geo_response = requests.get(geo_url, params=geo_params).json()
    
    if not geo_response.get("results"):
        return {"error": "Location not found"}
    
    result = geo_response["results"][0]
    lat, lon = result["latitude"], result["longitude"]

    # 2. Get weather (no API key needed)
    weather_url = "https://api.open-meteo.com/v1/forecast"
    weather_params = {
        "latitude": lat,
        "longitude": lon,
        "current_weather": True,
        "daily": "temperature_2m_max,temperature_2m_min,precipitation_sum,weathercode",
        "forecast_days": 3,
        "timezone": "auto"
    }
    
    weather_response = requests.get(weather_url, params=weather_params)
    if weather_response.status_code != 200:
        return {"error": "Failed to fetch weather data"}
    
    data = weather_response.json()
    return {
        "location": f"{result['name']}, {result.get('country', '')}",
        "current": data["current_weather"],
        "daily": data["daily"]
    }

In [18]:
weather_tool = {
    "type": "function",
    "function": {
        "name": "get_weather_forecast",
        "description": "Get weather forecast for a destination. Always use this when weather is relevant.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The destination city or region (e.g., 'Lake Tahoe', 'London')"
                }
            },
            "required": ["location"]
        }
    }
}

In [19]:
weather_agent = Assistants.create(
    model="qwen-plus",
    name="Weather Checker",
    description="Fetches weather forecasts for trip destinations using Open-Meteo API.",
    instructions="""
        You are a weather assistant. Your job is to provide accurate, up-to-date weather forecasts for travel destinations.
        You have access to the `get_weather_forecast` tool.

        Rules:
        1. ALWAYS use the `get_weather_forecast` tool when the user asks about weather or when planning a trip.
        2. The input to the tool must be the **destination** (e.g., 'Lake Tahoe', 'London').
        3. NEVER make up weather data — always use the tool.
        4. Present the forecast clearly: include high/low temps, conditions, and wind.
        5. Be concise and helpful.
    """,
        tools=[weather_tool]
    )

print(f"✅ {weather_agent.name} created with ID: {weather_agent.id}")

response = get_agent_response(
    weather_agent,
    "I want to go to London this weekend"
)

print(response)

✅ Weather Checker created with ID: asst_1f39e6c9-4eb5-4238-ab61-c9a461d909ce
Here’s the weather forecast for London this weekend:

- **Saturday (Jan 9)**:  
  High: 5.8°C | Low: 1.7°C  
  Conditions: Light rain  
  Wind: 11.9 km/h  

- **Sunday (Jan 10)**:  
  High: 5.8°C | Low: 1.7°C (weather likely similar)  

It will be cool and rainy — pack a warm jacket and a waterproof coat if you're planning to explore outdoors.


## 汇总所有信息：summary 智能体

In [20]:
summary_agent = Assistants.create(
    model="qwen-plus",
    name="Trip Concierge",
    description="Summarizes information from other agents into a final, polished trip plan for the user.",
    instructions="""
        You are TaskFriend, a friendly and helpful AI trip concierge.
        Your job is to take the information gathered by other agents and create a warm, clear, and actionable trip plan for the user.

        Input:
        - The original user query
        - The responses from: travel_agent, calendar_agent, weather_agent, maps_agent, etc.

        Rules:
        1. NEVER include raw JSON or technical details.
        2. Present the information in a structured, conversational format.
        3. Be encouraging and positive.
        4. Include key details: destination, availability, weather, travel time, and any recommendations.
        5. End with a helpful closing note.
    """
    )

print(f"✅ {summary_agent.name} created with ID: {summary_agent.id}")

✅ Trip Concierge created with ID: asst_c55e190e-dd10-4b0f-96ca-85dc5f3f6151


In [21]:
import ast
import re

def get_trip_plan_response(query: str):
    """
    End-to-end function to plan a trip.
    1. Uses planner_agent to decide the agent sequence.
    2. Executes each agent in order.
    3. Uses summary_agent to generate the final plan.
    """
    # 1. Get the plan from the planner_agent
    raw_plan = get_agent_response(planner_agent, query)
    print(f"🔍 Raw Planner Output: '{raw_plan}'")  # Debug: See what the planner actually returns
    
    # Robustly extract the list from the response
    try:
        # Remove any leading/trailing whitespace and newlines
        cleaned = raw_plan.strip()
        
        # Use regex to find the first list-like structure
        match = re.search(r'\[.*\]', cleaned)
        if not match:
            raise ValueError("No list found in planner response")
            
        list_str = match.group(0)
        agent_order = ast.literal_eval(list_str)
        
        print(f"🎯 Planner Agent decided on this workflow: {agent_order}")
        
    except Exception as e:
        print(f"❌ Failed to parse plan: {e}")
        # Fallback: use travel_agent if planning fails
        return get_agent_response(travel_agent, query)

    # 2. Map agent names to objects
    agent_mapper = {
        "travel_agent": travel_agent,
        "calendar_agent": calendar_agent,
        "weather_agent": weather_agent,
    }

    # 3. Execute each agent and collect their responses
    context = f"Original User Query: {query}\n\n"
    for agent_name in agent_order:
        agent = agent_mapper[agent_name]
        print(f"🛠️  Executing: {agent_name}")
        response = get_agent_response(agent, query)
        context += f"{agent_name} Response:\n{response}\n\n"

    # 4. Generate the final summary
    final_prompt = f"""
    Please create a final trip plan based on the following information:

    {context}

    Remember to be concise, friendly, and focus on the most important details.
    """
    final_response = get_agent_response(summary_agent, final_prompt)
    
    return final_response

In [22]:
result = get_trip_plan_response("我这个周末想去北京吃卤煮，从上海出发，希望周五走周日回，请规划一个省钱的行程.")

print(result)

🔍 Raw Planner Output: '["calendar_agent", "weather_agent", "travel_agent"]'
🎯 Planner Agent decided on this workflow: ['calendar_agent', 'weather_agent', 'travel_agent']
🛠️  Executing: calendar_agent
🛠️  Executing: weather_agent
🛠️  Executing: travel_agent
北京{
  "origin": "北京",
  "recommended_destinations": [
    {
      "name": "古北水镇",
      "travel_time": "2h drive",
      "reason": "This charming ancient water town combines traditional architecture with modern amenities, offering hot springs, scenic mountain views, and lantern-lit evening strolls—perfect for a relaxing yet culturally rich weekend escape just outside Beijing."
    },
    {
      "name": "承德避暑山庄",
      "travel_time": "2h 30m drive",
      "reason": "A UNESCO World Heritage Site, this vast imperial garden and palace complex provides a serene retreat into history and nature, ideal for exploring Qing dynasty heritage and enjoying peaceful lakeside walks over a weekend."
    },
    {
      "name": "首尔",
      "travel_tim

# 智能体开发平台

---

除了手写代码并自行设计智能体外，还有许多替代方案可以帮助你搭建智能体工作流。这些平台提供可视化界面、预置模板以及托管基础设施，加速智能体的开发与部署。

## Dify
[Dify](https://dify.ai/) 是一款开源平台，允许你通过可视化流程来构建 AI 智能体。它支持连接多种数据源、定义智能体行为，并将智能体部署到不同渠道。Dify 的优势在于灵活性与可扩展性，适合希望深度自定义智能体的开发者。

<div style="text-align: center;">
  <img src="images/LMP-C01_07-Dify GUI.gif" style="max-width: 800px;" />
  <br>
  <small>Dify 的可视化流程界面</small>
  <br>
  <small><i>来源：<a href="https://dify.ai" target="_blank">dify.ai</a></i></small>
</div>

**核心特性：**

* 可视化工作流构建器
* 支持多个 LLM 供应商
* 可通过插件系统扩展功能
* 提供 API，便于与外部系统集成
* 开源且支持自托管

> Dify 社区版也已在阿里云云效服务市场上架，提供两种形态：
> * [社区版](https://computenest.console.aliyun.com/service/detail/ap-southeast-1/service-1b86eaaec713493da092/7?type=user&isRecommend=true)
> * [企业版](https://computenest.console.aliyun.com/service/detail/ap-southeast-1/service-6d96d581dfaf43d3af1e/3?type=user&isRecommend=true)

## Hugging Face Agents
Hugging Face 提供了一整套构建与部署 AI 智能体的工具。它们与庞大的模型库深度整合，让你能够利用最前沿的模型。Hugging Face Agents 在自然语言处理任务方面尤其出色，并且可以轻松嵌入现有应用。

**核心特性：**

* 访问数以千计的预训练模型
* 提供便捷的部署选项
* 与主流框架集成
* 拥有活跃的社区支持
* 提供模型评估工具

## LangChain
LangChain 是一个用于开发语言模型驱动应用的框架。虽然它本身不是平台，但提供了构建复杂智能体系统所需的各类模块。LangChain 在把 LLM 与外部数据源和工具连接方面表现突出，因此非常适合需要与真实世界互动的智能体。

**核心特性：**

* 面向智能体开发的模块化组件
* 接入多种数据源与 API
* 支持多个 LLM 供应商
* 提供提示词管理与优化工具
* 拥有活跃社区与详实文档

## 如何选择合适的平台

挑选智能体开发平台时，可以从以下维度考虑：

* **用例复杂度：** 简单工作流可优先考虑 Dify 等可视化平台；复杂的多步骤流程可能需要 LangChain 这类更灵活的框架。
* **研发资源：** 如果你的团队拥有强大的研发能力，开源框架能提供更多自主控制；若技术资源有限，则可选择拥有可视化界面的托管平台。
* **集成需求：** 考虑智能体需要如何与现有系统和数据源交互。
* **可扩展性：** 评估平台能否应对未来的访问量和增长需求。
* **预算约束：** 部分平台提供免费层或开源版本，另一些则需要订阅费用。

平台的选择应与团队的技术能力、项目需求和长期目标相匹配。许多组织会先使用更简单的平台进行原型验证，再随着需求演进迁移到更成熟的框架。

# 接下来呢？

---

## 小测验

<details>
<summary style="cursor: pointer; padding: 12px; border: 1px solid #dee2e6; border-radius: 6px;">
<b>1. 在多智能体系统中，Planner 智能体的主要职责是什么？</b>  

<ul>
    <li>A) 生成最终的自然语言回答  </li>
    <li>B) 决定要调用哪些工具或子智能体，以及执行顺序  </li>
    <li>C) 存储用户偏好的长期记忆  </li>
    <li>D) 在文档上执行向量相似度检索</li>
</ul>

**查看答案 →**
</summary>

<div style="margin-top: 10px; padding: 15px; border: 1px solid #dee2e6; border-radius: 0 0 6px 6px;">

✅ **正确答案：** B) 决定要调用哪些工具或子智能体，以及执行顺序  
📝 **解析：**
* Planner 智能体负责拆解目标并进行编排——它会确定完成用户请求所需的行动顺序（例如：检查日历 → 查询天气 → 推荐目的地）。

</div>
</details>

## 核心要点

* **为什么要构建智能体系统？**
    * **RAG 负责回答问题——智能体负责解决问题**。
    * **真实世界的任务需要多步骤、工具与决策**——智能体能够统筹这些环节。
    * **智能体将复杂任务拆分为可管理的子任务**（例如：检查日历、获取天气）。
    * **它们支持自治、目标驱动的行为**——这是 AI 助手的下一个层级。

<br>

* **智能体编排**
    * **使用 Planner 智能体** 来协调子智能体（例如日历、天气等）。
    * **实现意图分类**，将查询路由到正确的智能体。
    * **串联智能体**：一个智能体的输出可以成为另一个的输入。
    * **保持模块化设计**——让每个智能体都专注完成一件事。
    * **编排可确保多步骤工作流的一致性与连贯性。**

<br>

* **工具集成**
    * **智能体依靠工具与外部系统交互**（API、日历、数据库等）。
    * **将函数封装成工具**（如 `function_tool`），便于智能体调用。
    * **始终包含错误处理**——智能体需要优雅地应对异常。
    * **工具让智能体能力超越上下文限制。**
    * **优秀的工具设计至关重要**——输入清晰，输出结构化。